In [7]:
from fastapi import FastAPI
from llama_index.embeddings.sagemaker_endpoint import SageMakerEmbedding
from llama_index.core import StorageContext, Document, VectorStoreIndex
from llama_index.vector_stores.weaviate import WeaviateVectorStore
import weaviate
from llama_index.core.memory import ChatMemoryBuffer
from llama_index.core.node_parser import SimpleNodeParser

from llama_index.llms.ollama import Ollama

c:\Users\abdel\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
app = FastAPI()


client = weaviate.Client("http://localhost:8080")

AWS_ACCESS_KEY_ID = ""
AWS_SECRET_ACCESS_KEY = ""
REGION_NAME = "eu-central-1"

# Initialize Weaviate vector store
vector_store = WeaviateVectorStore(
    weaviate_client=client, index_name="LlamaIndex"
)

# Initialize embedding model
embed_endpoint_model = SageMakerEmbedding(
    endpoint_name="BGE-M3",
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    region_name=REGION_NAME,
)

# Initialize storage context
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Initialize index
index = None



c:\Users\abdel\AppData\Local\Programs\Python\Python310\lib\site-packages\weaviate\warnings.py:121: DeprecationWarning: Dep005: You are using weaviate-client version 3.26.2. The latest version is 4.5.7.
            Please consider upgrading to the latest version. See https://weaviate.io/developers/weaviate/client-libraries/python for details.
  warnings.warn(


In [15]:
@app.post("/index-data/")
async def index_data(data: list[dict]):
    global index
    
    documents = [Document(**doc) for doc in data]
    parser = SimpleNodeParser.from_defaults(chunk_size=1024)
    nodes = parser.get_nodes_from_documents(documents)
    index = VectorStoreIndex(
            nodes, storage_context=storage_context, embed_model=embed_endpoint_model
        )

        #index.add_documents(documents)
    
    return {"message": "Data indexed successfully"}


@app.post("/query-data/")
async def query_data(query: str):
    if index is None:
        return {"error": "Index is not initialized. Please index data first."}
    
    llm = Ollama(base_url="https://n3iipm5bkl.execute-api.eu-central-1.amazonaws.com", model="mistral")
    query_engine = index.as_query_engine(llm=llm)
    response = query_engine.query(query)
    
    return response


@app.post("/chat-with-data/")
async def chat_data(query: str):
    if index is None:
        return {"error": "Index is not initialized. Please index data first."}
    
    llm = Ollama(base_url="https://n3iipm5bkl.execute-api.eu-central-1.amazonaws.com", model="mistral")
    memory = ChatMemoryBuffer.from_defaults()
    query_engine = index.as_query_engine(llm=llm,memory=memory)
    response = query_engine.query(query)
    
    return response


In [16]:
import asyncio
import uvicorn

if __name__ == "__main__":
    config = uvicorn.Config(app)
    server = uvicorn.Server(config)
    await server.serve()

INFO:     Started server process [19312]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:55062 - "POST /index-data/ HTTP/1.1" 200 OK
INFO:     127.0.0.1:55072 - "POST /query-data/?query=Can%20you%20list%20the%20names%20of%20all%20individuals%3F HTTP/1.1" 200 OK
INFO:     127.0.0.1:55073 - "POST /query-data/?query=What%20are%20the%20occupations%20of%20individuals%20from%20New%20York%3F HTTP/1.1" 200 OK
INFO:     127.0.0.1:55086 - "POST /query-data/?query=Who%20is%20the%20youngest%20person%20%3F HTTP/1.1" 500 Internal Server Error


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "c:\Users\abdel\AppData\Local\Programs\Python\Python310\lib\site-packages\httpx\_transports\default.py", line 69, in map_httpcore_exceptions
    yield
  File "c:\Users\abdel\AppData\Local\Programs\Python\Python310\lib\site-packages\httpx\_transports\default.py", line 233, in handle_request
    resp = self._pool.handle_request(req)
  File "c:\Users\abdel\AppData\Local\Programs\Python\Python310\lib\site-packages\httpcore\_sync\connection_pool.py", line 216, in handle_request
    raise exc from None
  File "c:\Users\abdel\AppData\Local\Programs\Python\Python310\lib\site-packages\httpcore\_sync\connection_pool.py", line 196, in handle_request
    response = connection.handle_request(
  File "c:\Users\abdel\AppData\Local\Programs\Python\Python310\lib\site-packages\httpcore\_sync\connection.py", line 99, in handle_request
    raise exc
  File "c:\Users\abdel\AppData\Local\Programs\Python\Python310\lib\site-pack

INFO:     127.0.0.1:55087 - "POST /query-data/?query=Who%20is%20the%20youngest%20individual%20%3F HTTP/1.1" 200 OK
INFO:     127.0.0.1:55096 - "POST /query-data/?query=Who%20is%20the%20youngest%20individual%20%3F HTTP/1.1" 200 OK
INFO:     127.0.0.1:55115 - "POST /query-data/?query=List%20all%20individuals%20over%20the%20age%20of%2030%20 HTTP/1.1" 200 OK
INFO:     127.0.0.1:55125 - "POST /chat-with-data/?query=Who%20is%20the%20software%20engineer%20from%20New%20York%3F HTTP/1.1" 200 OK
INFO:     127.0.0.1:55127 - "POST /chat-with-data/?query=How%20old%20is%20John%20Doe%3F HTTP/1.1" 200 OK
